# Definiton of all measurements

## Denition of KeyPoints

In [ ]:
# 'nose',                                   pos 0
#'left_eye', 'right_eye',                   pos 1 / 2
# 'left_ear', 'right_ear',                  pos 3 / 4
# 'left_shoulder', 'right_shoulder',        pos 5 / 6
# 'left_elbow', 'right_elbow',              pos 7 / 8
# 'left_wrist', 'right_wrist',              pos 9 / 10
# 'left_hip', 'right_hip',                  pos 11 / 12
# 'left_knee', 'right_knee',                pos 13 / 14
# 'left_ankle', 'right_ankle',              pos 15 / 16

#Format of 17 keypoints matrix for 1 image: 
#17 / 3 [y,x,likelyhood] 


In [4]:
import numpy as np
import math

In [ ]:
#ADDITIONAL KEYPOINTS

#left biceps, right biceps                                pos 17 / 18

def biceps_kp(image_front_T_kp, matrix_kp):
    #left biceps pos 17
    l_x = abs(image_front_T_kp[7][1] - image_front_T_k[5][1])
    l_y = (image_front_T_k[7][0] + image_front_T_k[5][0])/2
    #right biceps pos 18
    r_x = abs(image_front_T_k[8][1]-image_front_T_k[6][1])
    r_y = (image_front_T_k[8][0] + image_front_T_k[6][0])/2
    
    #append new keypoints into matrix_kp
    new_matrix_kp = np.append(matrix_kp, [l_y, l_x, 0], [r_y, r_x, 0], axis=0)
    return new_matrix_kp

#left chest, right chest                                pos 19 / 20
def chest_kp(image_front_T_kp, matrix_kp):
    #left chest pos 19 
    l_x = image_front_T_kp[5][1] + abs(image_front_T_kp[6][1] - image_front_T_kp[5][1]) * 1/4   #considering chest "x" located  at 1/4 vs soulders kp 5
    l_y = image_front_T_kp[5][0] + abs(image_front_T_kp[11][0] - image_front_T_kp[5][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips
    #right chest pos 20
    r_x = image_front_T_kp[5][1] + abs(image_front_T_kp[6][1] - image_front_T_kp[5][1]) * 3/4   #considering chest "x" located  at 3/4 vs soulders kp 5
    r_y = image_front_T_kp[6][0] + abs(image_front_T_kp[6][0] - image_front_T_kp[12][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips

    #append new keypoints into matrix_kp
    new_matrix_kp = np.append(matrix_kp, [l_y, l_x, 0], [r_y, r_x, 0], axis=0)
    return new_matrix_kp



#Convert Matrix Keypoints coordinates in x,y to pixels(contouring matrix)
def convert_kp_matrix_front(matrix_kp_front, image_h, image_l):
    x = np.round(matrix_kp_front[: , 1] * image_l)
    y = np.round(matrix_kp_front[: , 0] * image_h)
    matrix_kp_converted = list(zip(y,x))
    return matrix_kp_converted

def convert_kp_matrix_profile(matrix_kp_profile, image_h, image_l):
    x = np.round(matrix_kp_profile[: , 1] * image_l)
    y = np.round(matrix_kp_profile[: , 0] * image_h)
    matrix_kp_converted = list(zip(y,x))
    return matrix_kp_converted

# Define ratio between real measure(in mm) vs pixel position
def ratio_real_vs_pixel(real_height, matrix_image_contouring, image_h):
    list_sum_y=[]
    #sum of all "1" for each x to get the sum max ie. height
    for i in range(0,image_h):
        list_sum_y.append(np.sum(matrix_image_contouring[: , i]))
    ratio_mm_px = real_height / max(list_sum_y)                            #ratio real height (in mm) / max sum of "1"
    return ratio_mm_px

ratio_mm_px = def ratio_real_vs_pixel(real_height, matrix_image_contouring, image_h)


## Measurements for T-shirt
https://freesewing.org/designs/teagan/

In [ ]:
#Front picture in T shape
def biceps_circumference(matrix_kp_front_converted, matrix_image_front_cont):
    #left biceps
    x1 = matrix_kp_conv[17][1]
    sum_left = np.sum(matrix_image_front_cont[:, x1])
    
    #right biceps
    x2 = matrix_kp_conv[18][1]
    sum_right = np.sum(matrix_image_front_cont[:, x2])
    
    #diameter left / right biceps "height"
    average_mm = ((sum_left + sum_right) / 2) * ratio_mm_px
    #biceps circumference in mm
    circumference_biceps = (average_mm) * math.pi
    
    return  circumference_biceps

In [ ]:
#Front picture in T shape + profile picture
def chest_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont, ):
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[19][1]
    y_front2 = matrix_kp_front_converted[20][1]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_front_converted[19][1]
    y_prof2 = matrix_kp_front_converted[20][1]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #chest circumference in mm
    circumference_chest = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return circumference_chest

In [ ]:
#To be reviewed
#Current solution is distance between shoulder (# 5 / 6) and waist (# 21 / 22) with a factor 1.2
def hps_to_waist_back(matrix_kp_front_converted, matrix_image_front_cont):
    a = (matrix_kp_front_converted[5][0] - matrix_kp_front_converted[21][0]) + (matrix_kp_front_converted[6][0] - matrix_kp_front_converted[22][0])/2
    #convert in mm and apply factor of 1.2
    distance = a * 1.2 * ratio_mm_px
    
    return distance
    

In [ ]:
#Front picture in T shape + Profile picture
def hips_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont):
    
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[11][1]
    y_front2 = matrix_kp_front_converted[12][1]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_front_converted[11][1]
    y_prof2 = matrix_kp_front_converted[12][1]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #hips circumference in mm
    hips_circ = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return hips_circ

In [ ]:
#Profile picture
def neck_circumference( matrix_kp_profile_converted, matrix_image_profile_cont):
    #Estimated neck position on y located at 1/3 above shoulders vs ears
    neck_y = abs(matrix_kp_profile_converted[5][0] - matrix_kp_profile_converted[3][0])
    #sum over x considering neck_y = a
    sum_x = np.sum(matrix_image_profile_cont[neck_y,:])
    
    #neck circumference in mm
    neck_circ = (sum_x * ratio_mm_px) * math.pi
    
    return neck_circ
    

In [ ]:
#Front picture in T shape
#Consider distance (on x) between shoulder kp and ear kp, plus half width of biceps
def shoulder_slope(matrix_kp_front_converted, matrix_image_front_cont):
    #left biceps
    x1 = matrix_kp_front_converted[17][1]
    sum_left = np.sum(matrix_image_front_cont[:, x1])
    
    #distance (on x) between kp for shoulder / ear
    a = abs(matrix_kp_front_converted[5][1] - matrix_kp_front_converted[3][1]) + abs(matrix_kp_front_converted[4][1] - matrix_kp_front_converted[6][1]) / 2
    
    #overall distance in mm 
    total_distance = distance + sum_left/2 * ratio_mm_px
    
    return total_distance


In [ ]:
#Front picture in I shape
def shoulder_to_shoulder( matrix_kp_front_converted, matrix_image_front_cont ):
    #average y for shoulders kp
    x1 =( matrix_kp_front_converted[5][1] + matrix_kp_front_converted[6][1] ) / 2
    #distance (on x) between shoulders 
    sum_shoulders = np.sum(matrix_image_front_cont[x1, :])
    #convert distance in mm
    distance = sum_shoulders * ratio_mm_px
    
    return distance 
    

In [ ]:
#Front picture in T shape + Profile picture
def waist_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont):
    
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[21][0]
    y_front2 = matrix_kp_front_converted[22][0]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_profile_converted[11][0]
    y_prof2 = matrix_kp_profile_converted[12][0]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #waist circumference in mm
    waist_circumference = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return waist_circumference


In [ ]:
#Front picture in T shape
def waist_to_hips(matrix_kp_front_converted, matrix_image_front_cont):
    #average distance (on y) between waist(21,22) and hips(11,12) with correction factor of 1.2
    a = abs(matrix_kp_front_converted[22][1] - matrix_kp_front_converted[12][1])+abs(matrix_kp_front_converted[21][1] - matrix_kp_front_converted[11][1]) / 2
    #convert in mm and apply factor of 1.2
    distance = a * 1.2 * ratio_mm_px
    
    return distance


In [1]:
A = [[0.13626342, 0.57170576, 0.5406235 ],
       [0.12581952, 0.58031356, 0.5950187 ],
       [0.12306002, 0.5641533 , 0.5945765 ],
       [0.1330445 , 0.58676374, 0.46940276],
       [0.1319304 , 0.55489534, 0.48606545],
       [0.19684711, 0.6048043 , 0.37372732],
       [0.19630507, 0.53479004, 0.30953246],
       [0.26008302, 0.6065161 , 0.42267993],
       [0.25835255, 0.51460385, 0.49753478],
       [0.23563783, 0.59914243, 0.28201967],
       [0.25673127, 0.49728453, 0.4280582 ],
       [0.3964715 , 0.5841227 , 0.5168887 ],
       [0.3921252 , 0.54193866, 0.47156578],
       [0.5595846 , 0.58796036, 0.54856455],
       [0.5583141 , 0.54261553, 0.52169836],
       [0.7302867 , 0.5933529 , 0.4042293 ],
       [0.7241063 , 0.5598221 , 0.30607915]]


In [44]:
m = np.array([(1.5,2,3), (4,5,6), (10,-8,-4), (12,-4,-1)])
m

array([[ 1.5,  2. ,  3. ],
       [ 4. ,  5. ,  6. ],
       [10. , -8. , -4. ],
       [12. , -4. , -1. ]])

In [45]:
m[1,:]

array([4., 5., 6.])

In [41]:
sum(m[:,1])

-5.0

In [35]:
m.shape[0]

4

In [36]:
bb = range(0,11)
bb

range(0, 11)

In [37]:
for i in bb:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
